# Module

In [1]:
import sys

In [2]:
sys.path.remove('/home/jungeun/.local/lib/python3.6/site-packages')

In [3]:
import torch

In [4]:
sys.path.append('/home/jungeun/.local/lib/python3.6/site-packages')

In [5]:
import argparse
from tqdm import tqdm_notebook as tq
import tqdm
import os, time, math, copy
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from collections import namedtuple
import matplotlib.pyplot as plt

torch.set_printoptions(precision=8, linewidth=50000)
import warnings
warnings.filterwarnings(action='ignore')

# Print Colors

In [6]:
BLACK	= '\033[30m'
RED		= '\033[31m'
GREEN	= '\033[32m'
YELLOW	= '\033[33m'
BLUE	= '\033[34m'
MAGENTA	= '\033[35m'
CYAN	= '\033[36m'
RESET	= '\033[0m'
SEL		= '\033[7m'

In [7]:
class fxp:
    def __init__(self, bIn, iBWF):
        self.iFullBW = len(bIn)
        self.iIntgBW = self.iFullBW - iBWF
        self.bSign   = bIn[0]
        self.bIntg   = bIn[:self.iIntgBW]
        self.bFrac   = bIn[self.iIntgBW:]
        self.fFull   = 0
        try:
            for idx, bit in enumerate(bIn):
                if idx == 0:
                    self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
                else:
                    self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
        except:
            print(bIn)
        self.dispFull = RED + self.bIntg + BLUE + self.bFrac + RESET
        return

In [8]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol = 2 ** -iBWF
        #if fIn < self.fMin or fIn > self.fMax:
            #print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

        self.iFLP2INT = abs(int(fIn * 2 ** iBWF))
        if fIn < 0:
            self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

        if fIn >= 0:
            self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
        else:
            self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
            if len(self.bFull) > iBW:
                self.bFull = '0' * iBW

        self.cssFxp = fxp(self.bFull, self.iBWF)
        self.bSign = self.cssFxp.bSign
        self.bIntg = self.cssFxp.bIntg
        self.bFrac = self.cssFxp.bFrac
        self.fFull = self.cssFxp.fFull
        return

In [9]:
def flp2fixTensor(fIn, iBW, iBWF,args):
    fMin = - 2 ** (iBW - iBWF - 1)
    fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
    fList = []
    for aTensor in fIn.view(-1):
        fList.append(flp2fix(aTensor, iBW, iBWF).fFull)
    return torch.tensor(fList).view(fIn.size())
    fList = []
    for aTensor in fIn.view(-1):
        fList.append(flp2fix(aTensor, iBW, iBWF).fFull)
    return (torch.tensor(fList).view(fIn.size())).to(args.device)

# User Define Variables

In [10]:
data_path = '~/dataset'

In [11]:
parser = argparse.ArgumentParser(description='PyTorch for MNIST dataset')
parser.add_argument('--device', type=str, default='cuda', help='Device')
parser.add_argument('--shuffle', action='store_true', default=False, help='enables data shuffle')
parser.add_argument('--dataset', type=str, default='mnist', help='training dataset')
parser.add_argument('--data_path', type=str, default=data_path, help='path to MNIST')
parser.add_argument('--batch_size', type=int, default=64, help='batch size')
parser.add_argument('--epochs', type=int, default=10, help='number of epochs to train')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--optimizer', type=str, default='adam', help='optimizer')
parser.add_argument('--loss_func', type=str, default='cel', help='optimizer')
parser.add_argument('--quant_opt', type=str, default='asym', help='Type of Quantization')
parser.add_argument('--full_bits', type=int, default=16, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--pretrained', type=bool, default=True, help='Pretrained Model')
parser.add_argument('--act_quant', type=bool, default=False, help='Activation Quantization')
parser.add_argument('--disp', type=bool, default=False, help='Display Model Information')

args = parser.parse_args(args=[])

# Preparing Data

In [28]:
kwargs = {'num_workers': 1, 'pin_memory': True} if args.device == 'cuda' else {}
if args.dataset == 'mnist':
    train_loader = torch.utils.data.DataLoader(
        dataset=datasets.MNIST(
            root=args.data_path,
            train=True,
            download=True,
            transform=transforms.ToTensor()
        ),
        batch_size=args.batch_size,
        shuffle=args.shuffle,
        **kwargs
    )

    test_loader = torch.utils.data.DataLoader(
        dataset=datasets.MNIST(
            root=args.data_path,
            train=False,
            download=True,
            transform=transforms.ToTensor()
        ),
        batch_size=args.batch_size,
        shuffle=args.shuffle,
        **kwargs
    )

# Build Model

In [13]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 32)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(32, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        logits = self.fc3(x)
        return logits

In [14]:
def genOptimizer(model, args):
    if args.optimizer == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
    if args.optimizer == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    return optimizer

def genLossFunc(args):
    if args.loss_func == 'cel':
        loss_func = nn.CrossEntropyLoss()
    return loss_func

In [15]:
def train(train_loader, model, epoch, args):
    model.train()
    loss_func = genLossFunc(args)
    optimizer = genOptimizer(model, args)
    max_batch_index = int(np.floor(len(train_loader.dataset)/args.batch_size))
    running_loss = 0
    for batch_index, (image, label) in enumerate(tq(train_loader, desc='Train', leave=False)):
        image, label = image.to(args.device), label.to(args.device)
        pred = model(image)
        loss = loss_func(pred, label)
        running_loss += loss.item()#*image.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1:<3d}: Avg. Loss: {running_loss/len(train_loader.dataset):.4f}', end = '\t')

In [29]:
def test(test_loader, model, args):
    model.eval()
    with torch.no_grad():
        loss_func = genLossFunc(args)
        loss, correct = 0, 0
#  for batch_index, (image, label) in enumerate(tq(test_loader, desc='Test', leave=False)):
        for batch_index, (image, label) in enumerate(test_loader):
            image, label = image.to(args.device), label.to(args.device)
            pred = model(image)
            loss += loss_func(pred, label).item()#*image.size(0)
            correct += (pred.argmax(1) == label).type(torch.int).sum().item()
    loss /= len(test_loader.dataset)
    correct_rate = 100 * correct / len(test_loader.dataset)
    print(f'Accuracy: {correct}/{len(test_loader.dataset)} ({correct_rate:>.1f}%)')

In [30]:
def main(model):
    for epoch in range(args.epochs):
        train(train_loader, model, epoch, args)
        test(test_loader, model, args)
    print("Done!")
    return model

In [18]:
if args.pretrained:
    if os.path.isfile('preTrainedModel.pth'):
        model = MLP().to(args.device)
        model.load_state_dict(torch.load('preTrainedModel.pth'))
        test(test_loader, model, args)
    else:
        model = main(MLP().to(args.device))
        torch.save(model.state_dict(), 'preTrainedModel.pth')
else:
	model = main(MLP().to(args.device))
	torch.save(model.state_dict(), 'preTrainedModel.pth')

Accuracy: 9679/10000 (96.8%)


In [19]:
def model2fix(model, args):
	for name, _ in model.named_parameters():
		exec(f'model.{name}.data = flp2fixTensor(model.{name}.data, {args.full_bits}, {args.frac_bits}, args)')
	return model

In [20]:
def quantFixForward(model, x, args):
	cmodel = copy.deepcopy(model).to(args.device)
	with torch.no_grad():
		x = cmodel.flatten(x).to(args.device)
		act0 = flp2fixTensor(x, args.full_bits, args.frac_bits, args).to(args.device)
		
		act1 = model.relu1(cmodel.fc1(act0)).to(args.device)
		act1 = flp2fixTensor(act1, args.full_bits, args.frac_bits, args).to(args.device)

		act2 = cmodel.relu2(cmodel.fc2(act1)).to(args.device)
		act2 = flp2fixTensor(act2, args.full_bits, args.frac_bits, args).to(args.device)
		
		act3 = cmodel.fc3(act2).to(args.device)
		act3 = flp2fixTensor(act3, args.full_bits, args.frac_bits, args).to(args.device)
	return cmodel, act0, act1, act2, act3 

In [21]:
def testQuant(model, test_loader, args):
	qmodel = copy.deepcopy(model).to(args.device)
	qmodel = model2fix(qmodel, args)
	qmodel.eval()

	with torch.no_grad():
		loss_func = genLossFunc(args)
		loss, correct = 0, 0
		for batch_index, (image, label) in enumerate(tq(test_loader,desc='Test',leave=False)):
			image, label = image.to(args.device), label.to(args.device)
			qmodel, act0, act1, act2, act3  = quantFixForward(qmodel, image, args)
			y = act3
			loss += loss_func(y, label).item()#*image.size(0)
			correct += (y.argmax(1) == label).type(torch.int).sum().item()
	correct_rate = 100 * correct / len(test_loader.dataset)
	print(f'Accuracy: {correct}/{len(test_loader.dataset)} ({correct_rate:>.1f}%) Loss: {loss/len(test_loader.dataset):.2f}')
	return qmodel, act0, act1, act2, act3

In [22]:
qmodel, act0, act1, act2, act3 = testQuant(model, test_loader, args)

Test:   0%|          | 0/157 [00:00<?, ?it/s]

Accuracy: 9677/10000 (96.8%) Loss: 0.00


In [23]:
def extractParams(model, args):
	out_path = './mif'
	os.system(f'rm -rf {out_path};mkdir -p {out_path}')
	for key in model.state_dict().keys():
		layer_name = key.split('.')[0]
		param_type = 'w' if 'weight' in key else 'b'
		for idx, params in enumerate(eval(f'qmodel.{key}.data')):
			with open(f'{out_path}/{layer_name}_{param_type}_{idx}.mif', 'w') as fh:
				if param_type == 'w':
					for idx, param in enumerate(params):
						bin_param = flp2fix(param, args.full_bits, args.frac_bits).bFull
						fh.write(bin_param + ('\n','')[idx == len(params)-1])
				else:
					bin_param = flp2fix(params, args.full_bits, args.frac_bits).bFull
					fh.write(bin_param)
					
def genInputVector(test_loader, args):
	out_path = './vec'
	os.system(f'rm -rf {out_path};mkdir -p {out_path}')
	with open(f'{out_path}/labels.vec', 'w') as fh_labels:
		with open(f'{out_path}/images.vec', 'w') as fh_images:
			for batch_index, (images, labels) in enumerate(test_loader):
				for (image, label) in zip(images, labels):
					bin_label = flp2fix(label, args.full_bits, 0).bFull
					fh_labels.write(bin_label+'\n')
					for pixel in image.view(-1):
						bin_pixel = flp2fix(pixel, args.full_bits, args.frac_bits).bFull
						fh_images.write(bin_pixel+'\n')
                        

In [24]:
extractParams(model, args)
genInputVector(test_loader, args)

In [25]:
def genFPGAInput(test_loader, args):
	out_path = './bin'
	os.system(f'rm -rf {out_path};mkdir -p {out_path}')
	with open(f'{out_path}/labels.bin', 'wb') as fh_labels:
		with open(f'{out_path}/images.bin', 'wb') as fh_images:
			for batch_index, (images, labels) in enumerate(test_loader):
				for (image, label) in zip(images, labels):
					bin_label = flp2fix(label, args.full_bits, 0).bFull
					fh_labels.write(bytes([int(bin_label[8:], 2)]))
					fh_labels.write(bytes([int(bin_label[:8], 2)]))                    
					for pixel in image.view(-1):
						bin_pixel = flp2fix(pixel, args.full_bits, args.frac_bits).bFull
						fh_images.write(bytes([int(bin_pixel[8:], 2)]))
						fh_images.write(bytes([int(bin_pixel[:8], 2)]))

In [26]:
genFPGAInput(test_loader, args)

In [27]:
for mifFile in os.listdir('./mif'):
    binPath = './bin'
    mifPath = './mif'
    binFile = mifFile.split('.')[0] + '.bin'
    with open(f'{binPath}/{binFile}', 'wb') as fh_bin:
        with open(f'{mifPath}/{mifFile}', 'r') as fh_mif:
            for line in fh_mif.read().splitlines():
                fh_bin.write(bytes([int(line[8:], 2)]))
                fh_bin.write(bytes([int(line[:8], 2)]))        